In [5]:
!pip install pyldavis
!pip install gensim pyLDAvis
! python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyLDAvis-3.3.1-py2.py3-none-any.whl
  Using cached gensim-4.2.0-cp38-cp38-win_amd64.whl (24.0 MB)

  DEPRECATION: future is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Using cached numexpr-2.8.4-cp38-cp38-win_amd64.whl (92 kB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)
  Running setup.py install for future: started
  Running setup.py install for future: finished with status 'done'
Defaulting to user installation because normal site-packages is not writeable


Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


In [7]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.5/1.5 MB 1.9 MB/s eta 0:00:00
     -------------------------------------- 267.7/267.7 kB 2.4 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.2/12.2 MB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 181.6/181.6 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 96.7/96.7 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 481.3/481.3 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.0.1
    Uninstalling typing_exten

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
import pandas as pd
import numpy as np
import os

import string
import spacy
import nltk
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models

nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy.cli
spacy.cli.download('en_core_web_md')
import en_core_web_md

nlp = spacy.load('en_core_web_md')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GT\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GT\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [10]:
root_dir = r"C:\Users\GT\Documents\python\read_docx_file"
file_name = 'yelp.csv'

In [13]:
os.chdir(root_dir)

In [14]:
yelp_review = pd.read_csv(file_name)
yelp_review.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [16]:
print("Total Reviews")
print(len(yelp_review))
print("Unique Business")
print(len(yelp_review.groupby('business_id')))
print("Unique User")
print(len(yelp_review.groupby('user_id')))

Total Reviews
10000
Unique Business
4174
Unique User
6403


In [17]:
# clean the document and remove punctuation
def clean_text(text):
    delete_dict = {sp_char: '' for sp_char in string.punctuation}
    delete_dict[' '] =' '
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if (not w.isdigit() and ( not w.isdigit() and len(w)>3))])
    return text2.lower()

In [35]:
yelp_review['text_new'] = yelp_review['text'].apply(clean_text)
yelp_review['Num_words_text'] = yelp_review['text_new'].apply(lambda x:len(str(x).split()))

In [40]:
print('-------Reviews By Stars --------')
print(yelp_review['stars'].value_counts())
print(len(yelp_review))
print('-------------------------')
max_review_data_sentence_length  = yelp_review['Num_words_text'].max()

-------Reviews By Stars --------
4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64
10000
-------------------------


In [42]:
 # print short review (
mask = (yelp_review['Num_words_text'] < 100) & (yelp_review['Num_words_text'] >=20)
df_short_reviews = yelp_review[mask]
df_sampled = df_short_reviews.groupby('stars').apply(lambda x: x.sample(n=100)).reset_index(drop = True)
 
print('No of Short reviews')
print(len(df_short_reviews))

No of Short reviews
6276


In [45]:
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

In [46]:
# remove stopwords from the text
stop_words = stopwords.words('english')
df_sampled['text_new']=df_sampled['text_new'].apply(remove_stopwords)

In [58]:
# perform Lemmatization
lp = en_core_web_md.load(disable=['parser', 'ner'])
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']):
    output = []
    for sent in texts:
        doc = nlp(sent)
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
    return output

In [48]:
text_list=df_sampled['text'].tolist()

In [59]:
print(text_list[2])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[2])

May have great food but I'll never know. Ordered delivery and the chef who doesn't speak English couldn't find my apartment complex even when I walked out to the street right next to the 101 far away. He said he was there but couldn't tell me where he was. Talked to him on the phone for ten minutes then hung up on me when I guess he got frustrated and wouldn't answer my calls back. Called to order food for my visiting family and was glad to find this place but that chef is awful. The owner apologized to me profusely and that was a nice gesture but waiting an hour for a guy who hangs up on you and can't find a place looking at the 101 exit is ridiculous. Don't use them for delivery ever, call the Breakfast Joynt. Their owner is awesome and delivers to you himself. Plus, he actually knows the area. Cannot express my frustration with this restaurant's service. Do not use EVER
['great', 'food', 'delivery', 'chef', 'apartment', 'complex', 'street', 'phone', 'minute', 'call', 'food', 'family

In [60]:
# convert to document term frequency:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [61]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

In [62]:
# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary,
                num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [63]:
# print lda topics with respect to each word of document
lda_model.print_topics()

[(0,
  '0.022*"good" + 0.020*"food" + 0.016*"time" + 0.014*"place" + 0.012*"service" + 0.010*"great" + 0.009*"nice" + 0.008*"pizza" + 0.006*"drink" + 0.006*"lunch"'),
 (1,
  '0.015*"time" + 0.013*"place" + 0.013*"great" + 0.009*"restaurant" + 0.008*"price" + 0.007*"dish" + 0.007*"several" + 0.006*"lot" + 0.006*"staff" + 0.006*"home"'),
 (2,
  '0.021*"place" + 0.018*"food" + 0.017*"good" + 0.010*"bad" + 0.010*"restaurant" + 0.008*"staff" + 0.008*"time" + 0.008*"few" + 0.008*"friend" + 0.008*"more"'),
 (3,
  '0.017*"place" + 0.014*"order" + 0.010*"good" + 0.008*"service" + 0.007*"well" + 0.007*"drink" + 0.007*"title" + 0.007*"little" + 0.006*"nice" + 0.006*"lot"'),
 (4,
  '0.014*"food" + 0.012*"good" + 0.012*"restaurant" + 0.011*"service" + 0.009*"place" + 0.009*"lot" + 0.009*"buffet" + 0.009*"day" + 0.008*"lunch" + 0.008*"bit"'),
 (5,
  '0.023*"place" + 0.015*"food" + 0.012*"time" + 0.010*"good" + 0.010*"service" + 0.009*"price" + 0.009*"location" + 0.009*"people" + 0.009*"sandwich" + 0

In [65]:
# calculate perplexity and coherence
print('Perplexity: ', lda_model.log_perplexity(doc_term_matrix,
                                                total_docs=10000)) 

Perplexity:  -6.720699287236979


In [68]:
# calculate coherence
coherence_model_lda = CoherenceModel(model=lda_model,
                                     texts=tokenized_reviews, dictionary=dictionary ,
                                     coherence='c_v')

In [69]:
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence: ', coherence_lda)

Coherence:  0.3128874006286301


In [72]:
pyLDAvis.enable_notebook()

In [79]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(df_sampled['text_new'])

# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(random_state=0)

In [81]:
# Now, we use pyLDA vis to visualize it
pyLDAvis.gensim_models.prepare(lda_tf, dtm_tf, tf_vectorizer)

AttributeError: 'CountVectorizer' object has no attribute 'token2id'